# pLTV Python Model

## TO DO LIST

1. Add all forecasting methods to model.
2. Add back testing to model.
3. Revisit loan size forecast, errors seem large.

#### Resources

Github repo
https://github.com/kliao-tala/pLTV

Fader & Hardie paper on sBG model
https://drive.google.com/file/d/1tfMiERon1HgWo8dDJddwSzwc_AXK7LCA/view?usp=sharing
https://faculty.wharton.upenn.edu/wp-content/uploads/2012/04/Fader_hardie_jim_07.pdf

YT tutorial to pull look data
https://www.youtube.com/watch?v=EKwtLBnwXHk&list=PLXwS3L4W3KR2fhnQa-sLyajPZ9-L00KGX&index=1

Looker Python SDK & API
https://inventure.looker.com/extensions/marketplace_extension_api_explorer::api-explorer/4.0

---

In [121]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from scipy.optimize import curve_fit, minimize

import plotly
from plotly import graph_objects as go
import plotly.io as pio

# change default plotly theme
pio.templates.default = "plotly_white"

import sbg

In [280]:
inputs = pd.read_csv('data/pltv_inputs.csv')
data = pd.read_csv('data/ke_data.csv')
pltv_expected = pd.read_csv('data/pltv_expected.csv')

## Data Munging

In [281]:
# fix date inconsistencies
data = data.replace({'2021-9': '2021-09', '2021-8': '2021-08', '2021-7': '2021-07', 
              '2021-6': '2021-06', '2021-5': '2021-05', '2021-4': '2021-04', '2020-9': '2020-09'})

# sort by months since first disbursement
data = data.sort_values(['First Loan Local Disbursement Month', 
                         'Months Since First Loan Disbursed'])

# remove all columns calculated through looker
data = data.loc[:,:"Default Rate Amount 51D"]

In [282]:
data.head()

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,Count Borrowers,Count Loans,Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,Default Rate Amount 30D,Default Rate Amount 51D
0,2020-09,0,7801,7801,13156,48361000,6540240,681325,81520,0.155382,0.121192,0.113031
15,2020-09,1,0,4481,5697,34490000,4660880,416544,32387,0.130661,0.101823,0.095738
2,2020-09,2,0,3661,4310,31461000,4297310,401077,30617,0.139719,0.103958,0.094792
9,2020-09,3,0,3050,3599,30482000,4178400,343062,17629,0.125111,0.089089,0.084399
10,2020-09,4,0,2549,2985,29303000,3964590,300262,920,0.113720,0.086750,0.081658


## pLTV Model

In [283]:
# KSH to USD conversion factor
ksh_usd = 0.00925
late_fee = 0.08 # % fee on defaults

In [318]:
class Model:
    """
    sBG model class containing all functionality for creating, analyzing, and backtesting
    the sBG model.
    
    Parameters
    ----------
    data : pandas DataFrame
        Raw data pulled from Looker.
        
    Methods
    -------
    clean_data
        Performs all data cleaning steps and returns the cleaned data.
        
    borrower_retention(cohort_data)
        Computes borrower retention.
    
    """
    
    def __init__(self, data):
        self.data = data

        self.clean_data()
        
        
    def clean_data(self):
        # fix date inconsistencies
        self.data = self.data.replace({'2021-9': '2021-09', '2021-8': '2021-08', \
                                       '2021-7': '2021-07', '2021-6': '2021-06', \
                                       '2021-5': '2021-05', '2021-4': '2021-04', \
                                       '2020-9': '2020-09'})

        # sort by months since first disbursement
        self.data = self.data.sort_values(['First Loan Local Disbursement Month', 
                                 'Months Since First Loan Disbursed'])

        # remove all columns calculated through looker
        self.data = self.data.loc[:,:"Default Rate Amount 51D"]
        
        # add more convenient cohort column
        self.data['cohort'] = self.data['First Loan Local Disbursement Month']
        
        
    # --- DATA FUNCTIONS --- #
    def borrower_retention(self, cohort_data):
        return cohort_data['Count Borrowers']/cohort_data['Count Borrowers'].max()

    
    def borrower_survival(self, cohort_data):
        return cohort_data['Count Borrowers']/cohort_data['Count Borrowers'].shift(1)
    
    
    def loans_per_borrower(self, cohort_data):
        return cohort_data['Count Loans']/cohort_data['Count Borrowers']
    
    
    def loan_size(self, cohort_data, to_usd):
        df = cohort_data['Total Amount']/cohort_data['Count Loans']
        if to_usd:
            df *= ksh_usd
        return df
    
    
    def interest_rate(self, cohort_data):
        return cohort_data['Total Interest Assessed']/cohort_data['Total Amount']
    
    
    def default_rate(self, cohort_data, period=7):
        if period==7:
            return cohort_data['Default Rate Amount 7D'].fillna(0)
        
        elif period==51:
            default_rate = cohort_data['Default Rate Amount 51D']

            recovery_rate_51 = float(inputs[inputs.market=='ke']['recovery_7-30'] + \
                                     inputs[inputs.market=='ke']['recovery_30-51'])

            ## fill null 51dpd values with 7dpd values based on recovery rates
            derived_51dpd = (cohort_data['Count Loans']*(cohort_data['default_rate_7dpd']) - \
                cohort_data['Count Loans']*(cohort_data['default_rate_7dpd'])*recovery_rate_51)/ \
                cohort_data['Count Loans']
            
            return default_rate.fillna(derived_51dpd)
        
        elif period==365:
            # get actual data if it exists
            default_rate = np.nan*cohort_data['Default Rate Amount 51D']

            recovery_rate_365 = float(inputs[inputs.market=='ke']['recovery_51_'])

            ## fill null 365dpd values with 51dpd values based on recovery rates
            derived_365dpd = (cohort_data['Count Loans']*(cohort_data['default_rate_51dpd']) - \
                cohort_data['Count Loans']*(cohort_data['default_rate_51dpd'])* \
                recovery_rate_365)/cohort_data['Count Loans']

            return default_rate.fillna(derived_365dpd)
        
        
    def loans_per_original(self, cohort_data):
        return cohort_data['Count Loans']/cohort_data['Count Borrowers'].max()
    
    
    def origination_per_original(self, cohort_data, to_usd):
        df = cohort_data['Total Amount']/cohort_data['Count Borrowers'].max()
        if to_usd:
            df *= ksh_usd
        return df
    
    
    def revenue_per_original(self, cohort_data, to_usd):
        interest_revenue = cohort_data['origination_per_original']*cohort_data['interest_rate']
        
        # 0.08 is the % fee we charge to defaulted customers
        revenue = interest_revenue + (cohort_data['origination_per_original'] + interest_revenue) * \
            cohort_data['default_rate_7dpd']*0.08
        
        # note that origination_per_original is already in USD so no conversion is necessary
        return revenue
    
    
    def credit_margin(self, cohort_data):
        return cohort_data['revenue_per_original'] - \
                (cohort_data['origination_per_original'] + cohort_data['revenue_per_original'])* \
                cohort_data['default_rate_365dpd']
    
    
    def opex_per_original(self, cohort_data):
        opex_cost_per_loan = float(inputs[inputs.market=='ke']['opex cost per loan'])
        cost_of_capital = float(inputs[inputs.market=='ke']['cost of capital'])/12
        
        return opex_cost_per_loan*cohort_data['loans_per_original'] + \
            cost_of_capital*cohort_data['origination_per_original']
    
    
    def ltv_per_original(self, cohort_data):
        return cohort_data['cm$_per_original'] - cohort_data['opex_per_original']
    
    
    def credit_margin_percent(self, cohort_data):
        return cohort_data['ltv_per_original']/cohort_data['revenue_per_original']
        
        
    def generate_features(self, to_usd=True):
        """
        Generate all features required for pLTV model.
        """
        cohorts = []

        # for each cohort
        for cohort in self.data.loc[:,'First Loan Local Disbursement Month'].unique():
            # omit the last two months of incomplete data
            cohort_data = self.data[self.data['First Loan Local Disbursement Month']==cohort].iloc[:-2,:]

            # call data functions to generate calculated features
            cohort_data['borrower_retention'] = self.borrower_retention(cohort_data)
            cohort_data['borrower_survival'] = self.borrower_survival(cohort_data)
            cohort_data['loans_per_borrower'] = self.loans_per_borrower(cohort_data)
            cohort_data['loan_size'] = self.loan_size(cohort_data, to_usd)
            cohort_data['interest_rate'] = self.interest_rate(cohort_data)
            cohort_data['default_rate_7dpd'] = self.default_rate(cohort_data, period=7)
            cohort_data['default_rate_51dpd'] = self.default_rate(cohort_data, period=51)
            cohort_data['default_rate_365dpd'] = self.default_rate(cohort_data, period=365)
            cohort_data['loans_per_original'] = self.loans_per_original(cohort_data)
            cohort_data['origination_per_original'] = self.origination_per_original(cohort_data, to_usd)
            cohort_data['revenue_per_original'] = self.revenue_per_original(cohort_data, to_usd)
            cohort_data['cm$_per_original'] = self.credit_margin(cohort_data)
            cohort_data['opex_per_original'] = self.opex_per_original(cohort_data)
            cohort_data['ltv_per_original'] = self.ltv_per_original(cohort_data)
            cohort_data['cm%_per_original'] = self.credit_margin_percent(cohort_data)
            
            # reset the index and append the data
            cohorts.append(cohort_data.reset_index(drop=True))

        self.cohorts = cohorts
        self.data = pd.concat(cohorts, axis=0)
    
    
    def plot_cohorts(self, param, forecast=False):
        """
        Generate scatter plot for a specific paramter.
        
        Parameters
        ----------
        
        """
        
        curves = []
        if forecast:
            for cohort in self.forecast.loc[:,'First Loan Local Disbursement Month'].unique():
                output = self.forecast[self.forecast['First Loan Local Disbursement Month']==cohort]

                output = output[param].reset_index(drop=True)
                output.name = cohort

                curves.append(output)
        else:
            for cohort in self.data.loc[:,'First Loan Local Disbursement Month'].unique():
                output = self.data[self.data['First Loan Local Disbursement Month']==cohort]

                output = output[param].reset_index(drop=True)
                output.name = cohort

                curves.append(output)
            
        traces = []

        for cohort in curves:
            traces.append(go.Scatter(name=cohort.name, x=cohort.index, y=cohort))

        fig = go.Figure(traces)
        fig.update_layout(xaxis=dict(title='Month Since Disbursement'),
                         yaxis=dict(title=param))

        fig.show()
        
        
    # --- FORECAST FUNCTIONS --- #
    def forecast_borrowers(self, months=24):
        """
        Generates a forecast of "Count Borrowers" out to the input number of months.
        The original and forecasted values are returned as a new dataframe, set as
        a new attribute of the model, *.forecast*. 
        
        Parameters
        ----------
        months : int
            Number of months to forecast to.
        """
        
        # initialize alpha and beta, optimized later by model
        alpha = beta = 1
        
        # list to hold individual cohort forecasts
        forecast_dfs = []

        # range of desired time periods
        times = list(range(1, months+1))
        
        for cohort in m.data.cohort.unique():
            # data for current cohort
            c_data = m.data[m.data.cohort==cohort]
            
            # starting cohort size
            n = c_data.loc[0, 'Count Borrowers']

            # only for cohorts with at least 4 data points
            if len(c_data) > 3:
                c = c_data['Count Borrowers']

                # define bounds for alpha and beta (must be positive)
                bounds = ((0,None), (0,None))
                
                # use scipy's minimize function on log_likelihood to optimize alpha and beta
                results = minimize(log_likelihood, np.array([alpha,beta]), bounds=bounds)

                # list to hold forecasted values 
                forecast = []
                for t in times:
                    forecast.append(n*s(t, results.x[0], results.x[1]))

                # convert list to dataframe
                forecast = pd.DataFrame(forecast, index=times, columns=['Count Borrowers'])

                # null df used to extend original cohort df to desired number of forecast months
                dummy_df = pd.DataFrame(np.nan, index=range(0,months+1), columns=['null'])

                # create label column to denote actual vs forecast data
                c_data['data_type'] = 'actual'

                # extend cohort df and ffill cohort labels
                c_data = pd.concat([c_data, dummy_df], axis=1).drop('null', axis=1)
                c_data.cohort = c_data.cohort.ffill()

                # label forecasted data
                c_data.data_type = c_data.data_type.fillna('forecast')

                # fill in the forecasted data
                c_data['Count Borrowers'] = c_data['Count Borrowers'].fillna(forecast['Count Borrowers'])
                
                # add retention & survival features
                c_data['borrower_retention'] = m.borrower_retention(c_data)
                c_data['borrower_survival'] = m.borrower_survival(c_data)
                
                
                
                # forecast loan size
                #if there's an actual, use actual, otherwise use last point*slope of pLTV curve
                for i in c_data[c_data.loan_size.isnull()].index:
                    c_data.loc[i, 'loan_size'] = c_data.loc[i-1, 'loan_size'] * \
                    pltv_expected.loc[i,'loan_size']/pltv_expected.loc[i-1, 'loan_size']
                
                
                forecast_dfs.append(c_data)

        self.forecast = pd.concat(forecast_dfs)
    
        
    def forecast(self):
        """
        Convenience function to generate forecasts on all features. Forecast data
        is returned as a separate dataframe and attached as an attribute to the 
        instantiated model.
        
        Parameters
        ----------
        
        Returns
        -------
        
        """
        
        pass

Review calculations for revenue per original, they're very off. Used a different formula than Liang's. Understand his formula.

#### Issues to fix

1. null values in default rates.
2. forecasting vs not, the final incomplete month.

### Run Model

In [311]:
# create a model object
m = Model(data)

# generate features
m.generate_features()

m.data.head()

,First Loan Local Disbursement Month,Months Since First Loan Disbursed,Count First Loans,Count Borrowers,Count Loans,Total Amount,Total Interest Assessed,Total Rollover Charged,Total Rollover Reversed,Default Rate Amount 7D,...,default_rate_7dpd,default_rate_51dpd,default_rate_365dpd,loans_per_original,origination_per_original,revenue_per_original,cm$_per_original,opex_per_original,ltv_per_original,cm%_per_original
0,2020-09,0,7801,7801,13156,48361000,6540240,681325,81520,0.155382,...,0.155382,0.113031,0.099467,1.686450,57.343834,8.564274,2.008581,2.847339,-0.838759,-0.097937
1,2020-09,1,0,4481,5697,34490000,4660880,416544,32387,0.130661,...,0.130661,0.095738,0.084249,0.730291,40.896359,6.011869,2.059888,1.407028,0.652860,0.108595
2,2020-09,2,0,3661,4310,31461000,4297310,401077,30617,0.139719,...,0.139719,0.094792,0.083417,0.552493,37.304737,5.569447,1.993030,1.133426,0.859604,0.154343
3,2020-09,3,0,3050,3599,30482000,4178400,343062,17629,0.125111,...,0.125111,0.084399,0.074271,0.461351,36.143892,5.365869,2.282891,1.000542,1.282349,0.238982
4,2020-09,4,0,2549,2985,29303000,3964590,300262,920,0.113720,...,0.113720,0.081658,0.071859,0.382643,34.745898,5.059868,2.199469,0.881503,1.317966,0.260474


In [312]:
m.forecast_borrowers()

In [317]:
m.forecast[m.forecast.cohort=='2020-12']['loan_size']

0      31.881044
1      52.877890
2      65.640610
3      79.973257
4      97.846539
5     113.875648
6     128.348180
7     140.995885
8     149.119292
9     159.121049
10    167.067712
11    174.859662
12    178.297332
13    183.116179
14    187.935026
15    191.790103
16    192.753873
17    192.753873
18    192.753873
19    192.753873
20    192.753873
21    192.753873
22    192.753873
23    192.753873
24    192.753873
Name: loan_size, dtype: float64

In [39]:
m.data.columns

Index(['First Loan Local Disbursement Month',
       'Months Since First Loan Disbursed', 'Count First Loans',
       'Count Borrowers', 'Count Loans', 'Total Amount',
       'Total Interest Assessed', 'Total Rollover Charged',
       'Total Rollover Reversed', 'Default Rate Amount 7D',
       'Default Rate Amount 30D', 'Default Rate Amount 51D', 'cohort',
       'borrower_retention', 'borrower_survival', 'loans_per_borrower',
       'loan_size', 'interest_rate', 'default_rate_7dpd', 'default_rate_51dpd',
       'default_rate_365dpd', 'loans_per_original', 'origination_per_original',
       'revenue_per_original', 'cm$_per_original', 'opex_per_original',
       'ltv_per_original', 'cm%_per_original'],
      dtype='object')

In [314]:
# visualize cohorts for a given feature
m.plot_cohorts('loan_size')

## Forecasting

### Power law regression

In [238]:
def power_law(x, A, B):
    return A*x**B

In [245]:
arr = m.data[m.data['First Loan Local Disbursement Month']=='2020-12'].loc[1:, 'Count Loans']
arr = arr.dropna()
power_param, power_cov = curve_fit(power_law, arr.index, arr)

In [246]:
x = list(range(1,25))
power_fit = power_law(x, power_param[0], power_param[1])

In [247]:
traces = [
    go.Scatter(name='actual', x=arr.index, y=arr),
    go.Scatter(name='power-law', x=x, y=power_fit)
]

fig = go.Figure(traces)

fig.show()

### sBG probalistic model

sBG model assumptions:
1. The propensity of one customer to drop out is independent of the behavior of every other customer.
2. Individual customer retention rates are unchanged over time.
3. Observed retention increase with time due to aggregate results and heterogenity in customer behaviors.
4. Model applies for customer relationships in "discrete-time" and "contractual" settings.

In [53]:
# initial guesses @ alpha and beta
alpha = 1
beta = 1

In [85]:
def p(t, alpha, beta):
    """
    Probability that a customer fails to take out another loan (probability to churn).
    For the derivation of this equation, see the original Fader & Hardie paper. This 
    recursion formula takes two constants, alpha and beta, which are fit to actual data.
    It then allows you to compute the probability of churn for a given time period, t.
    
    Parameters
    ----------
    t : int
        Time period.
    alpha : float
        Fitting parameter.
    beta : float
        Fitting parameter.
    
    Returns
    -------
    P : float
        Probability of churn.
    """
    
    if t==1:
        return alpha/(alpha + beta)
    else:
        return p(t-1, alpha, beta) * (beta+t-2)/(alpha+beta+t-1)
    
def s(t, alpha, beta):
    """
    Survival function: the probability that a customer has survived to time t.
    For the derivation of this equation, see the original Fader & Hardie paper. This 
    recursion formula takes two constants, alpha and beta, which are fit to actual data.
    It also requires computation of P (probability of a customer churning).
    
    Parameters
    ----------
    t : int
        Time period.
    alpha : float
        Fitting parameter.
    beta : float
        Fitting parameter.
    
    Returns
    -------
    P : float
        Probability of churn.
    """
    
    if t==1:
        return 1 - p(t, alpha, beta)
    else:
        return s(t-1, alpha, beta) - p(t, alpha, beta)
    
def log_likelihood(params):
    """
    Computes the *negative* log-likelihood of the probability distribution of customers
    still being active at time t. For a derivation of the log-likelihood, see Appendix A
    in the original Fader & Hardie paper. The function computes the log-likelihood at 
    every time step, t, leading up to the last time period T. The final value is simply
    the sum of the log-likelihood computed at each time step. In the end, we return the 
    negative of the log-likelihood so that we can use scipy's minimize function to optimize
    for the values of alpha and beta.
    
    
    Parameters
    ----------
    t : int
        Time period.
    alpha : float
        Fitting parameter.
    beta : float
        Fitting parameter.
    
    Returns
    -------
    ll : float
        log-likelihood value
    """
        
    alpha, beta = params
    
    # initialize log-likelihood (ll) value at 0
    ll=0
    
    # for each time period in the *actual* data, compute ll and add it to the running total
    for t in c[1:].index:
        ll += (c[t-1]-c[t])*np.log(p(t, alpha, beta))
    
    # add the final term which associated with customers who are still active at the end
    # of the final period.
    ll += c.iloc[-1]*np.log(s((len(c)-1)-1, alpha, beta)-p(len(c)-1, alpha, beta))
    
    return -ll

In [86]:
cohort = '2020-09'

In [87]:
c = m.data[m.data['First Loan Local Disbursement Month']==cohort]['Count Borrowers']
c = c.reset_index(drop=True)

In [88]:
# since we're working with logs, we need bounds for alpha and beta > 0.
bounds = ((0,None), (0,None))

results = minimize(log_likelihood, np.array([1,1]), bounds=bounds)
results

      fun: 15970.457182334316
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
      jac: array([-0.0001819, -0.0001819])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 36
      nit: 9
     njev: 12
   status: 0
  success: True
        x: array([0.71002432, 1.04180686])

In [89]:
alpha_opt, beta_opt = results.x

sBG_forecast = []
for i in x:
    sBG_forecast.append(s(i, alpha_opt, beta_opt))

In [90]:
arr = m.data[m.data['First Loan Local Disbursement Month']==cohort].loc[1:, 'borrower_retention']
arr = arr.dropna()

power_param, power_cov = curve_fit(power_law, arr.index, arr)
power_fit = power_law(x, power_param[0], power_param[1])

In [91]:
traces = [
    go.Scatter(name='actual', x=arr.index, y=arr),
    go.Scatter(name='power-law', x=x, y=power_fit),
    go.Scatter(name='sBG', x=x, y=sBG_forecast)
]

fig = go.Figure(traces)
fig.update_layout(xaxis=dict(title='Months Since First Loan'),
                 yaxis=dict(title='Retention (%)'))

fig.show()

In [92]:
power_res = abs(power_fit[:len(arr)] - arr)
sbg_res = abs(sBG_forecast[:len(arr)] - arr)

traces = [
    go.Scatter(name='power-law', x=arr.index, y=power_res),
    go.Scatter(name='sBG', x=arr.index, y=sbg_res)
]

fig = go.Figure(traces)
fig.update_layout(title='Model Residuals', xaxis=dict(title='Months Since First Loan'))

fig.show()

#### Scale forecast

In [118]:
forecast_dfs = []

alpha = beta = 1

x = list(range(1, 25))
df = m.data[['cohort', 'Count Borrowers']]
for cohort in df.cohort.unique():
    c_data = df[df.cohort==cohort]
    n = c_data.loc[0, 'Count Borrowers']
    
    # only for cohorts with at least 4 data points
    if len(c_data) > 3:
        c = c_data['Count Borrowers']

        # fit model
        bounds = ((0,None), (0,None))
        results = minimize(log_likelihood, np.array([alpha,beta]), bounds=bounds)

        # forecast
        forecast = []
        for i in x:
            forecast.append(n*s(i, results.x[0], results.x[1]))

        forecast = pd.DataFrame(forecast, index=x, columns=['Count Borrowers'])
    
        holder_df = pd.DataFrame(np.nan, index=range(0,25), columns=['null'])
        
        c_data['data_type'] = 'actual'
        
        c_data = pd.concat([c_data, holder_df], axis=1).drop('null', axis=1)
        c_data.cohort = c_data.cohort.ffill()

        c_data.data_type = c_data.data_type.fillna('forecast')
        
        c_data = c_data.fillna(forecast)
        
        c_data['borrower_retention'] = m.borrower_retention(c_data)
        
        forecast_dfs.append(c_data)
        
forecast = pd.concat(forecast_dfs)

In [120]:
cohort='2021-06'

df = forecast[forecast.cohort==cohort]

traces = []
for dtype in df.data_type.unique():
    traces.append(go.Scatter(name=dtype, x=df[df.data_type==dtype].index, 
                             y=df[df.data_type==dtype]['borrower_retention'], mode='markers+lines'))
    
fig = go.Figure(traces)
fig.update_layout(xaxis=dict(title='Months Since Disbursement'),
                 yaxis=dict(title='Count Borrowers'))

fig.show()

### Backtest Framework

In [95]:
def test(actual, forecast, metric='rmse'):
    """
    Test forecast performance against actuals using method defined by metric.
    """
    if metric=='rmse':
        error = np.sqrt(sum((forecast[:len(actual)] - actual)**2))
    elif metric=='mae':
        error = np.mean(forecast[:len(actual)] - actual)
    return error

In [96]:
test(arr, power_fit, metric='rmse') 

0.09916461969488528

In [97]:
test(arr, sBG_forecast)

0.0594282453045051

### Test all cohorts

In [99]:
test_vals = {}

for cohort in m.data.cohort.unique():
    x = list(range(1,25))

    # power-law
    arr = m.data[m.data.cohort==cohort]['borrower_retention'][1:].dropna()
    
    if len(arr)>=3:
        power_param, power_cov = curve_fit(power_law, arr.index, arr)
        power_fit = power_law(x, power_param[0], power_param[1])

        # sbg
        c = m.data[m.data.cohort==cohort]['Count Borrowers'].reset_index(drop=True)

        # since we're working with logs, we need bounds for alpha and beta > 0.
        bounds = ((0,None), (0,None))

        results = minimize(log_likelihood, np.array([1,1]), bounds=bounds)

        sBG_forecast = []
        for i in x:
            sBG_forecast.append(s(i, results.x[0], results.x[1]))

        test_vals[cohort] = {'power-law': test(arr, power_fit, metric='rmse'), 
                             'sbg': test(arr, sBG_forecast)}
    
test_vals

{'2020-09': {'power-law': 0.09916461969488528, 'sbg': 0.0594282453045051},
 '2020-10': {'power-law': 0.08878383231424848, 'sbg': 0.06802539463007341},
 '2020-11': {'power-law': 0.07419163319525678, 'sbg': 0.05531285348505579},
 '2020-12': {'power-law': 0.06790207359132192, 'sbg': 0.05201175246265755},
 '2021-01': {'power-law': 0.07911571109778857, 'sbg': 0.06382120180468376},
 '2021-02': {'power-law': 0.054426707243568566, 'sbg': 0.03835302216820122},
 '2021-03': {'power-law': 0.0776107847336467, 'sbg': 0.0692911975445195},
 '2021-04': {'power-law': 0.05722561737841853, 'sbg': 0.04604031032253486},
 '2021-05': {'power-law': 0.06282766865763831, 'sbg': 0.051682282453034134},
 '2021-06': {'power-law': 0.04683240567333825, 'sbg': 0.03902919592754487},
 '2021-07': {'power-law': 0.04955487707307277, 'sbg': 0.044694318959938195},
 '2021-08': {'power-law': 0.04515225858421964, 'sbg': 0.045536779858922405},
 '2021-09': {'power-law': 0.019865115724015205, 'sbg': 0.017385765475413713},
 '2021-10

In [ ]:
m.data[m.data.cohort=='2021-10']

In [ ]:
forecast_methods = {
    'Count Borrowers': 'sbg',
    'loans_per_borrwer': 'power-law'
}

In [ ]:
m.data[m.data['First Loan Local Disbursement Month']=='2020-09']['Count Borrowers']

#### Questions/Concerns

1. Is there a framework that's been developed/used to back test this forecast model?
2. Filter out bad cohorts. Why are some starting at 0? (e.g. 2021-07)
3. Why are the last few months not included? Why not just omit the final incomplete month?


In [ ]:
new_data[new_data['First Loan Local Disbursement Month'] == '2021-07']